In [1]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [2]:
token = '9593ca7b9d6cc91a9a2bebdd8ad743fb400d76ab3994b5c71016070993bd25b16ba5c31e0800898faaca5'

In [3]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

In [5]:
vk.messages.send(
    chat_id=1,
    random_id=1,
    message='Matrix has you ...')

3

In [6]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-novohatnij-29/shared/ads_data.csv.zip', compression='zip')

In [7]:
df

,ad_id,time,event,date,ad_cost_type,has_video,client_union_id,campaign_union_id,platform,ad_cost,target_audience_count,user_id
0,23456,1554076848,view,2019-04-01,CPM,0,5681,23456,android,190.0,125560,793
1,111941,1554135866,view,2019-04-01,CPM,0,111793,111941,ios,215.2,32277,1041
2,111941,1554135866,view,2019-04-01,CPM,0,111793,111941,ios,215.2,32277,41
3,42987,1554135866,view,2019-04-01,CPM,0,28541,42985,ios,199.1,80160,898
4,23456,1554135866,view,2019-04-01,CPM,0,5681,23456,web,190.0,125560,29
...,...,...,...,...,...,...,...,...,...,...,...,...
3176709,112583,1554501092,view,2019-04-06,CPM,0,111818,112260,android,199.4,239408,5467
3176710,112583,1554501092,view,2019-04-06,CPM,0,111818,112260,android,199.4,239408,2508
3176711,112583,1554501092,click,2019-04-06,CPM,0,111818,112260,web,199.4,239408,7896
3176712,112583,1554501092,view,2019-04-06,CPM,0,111818,112260,ios,199.4,239408,4319


In [8]:
ad_data = df.groupby(['ad_id', 'event'], as_index=False) \
    .agg({'user_id': 'count'})

In [9]:
ad_data = ad_data.pivot(index='ad_id', columns='event', values='user_id').reset_index()

In [10]:
ad_data = ad_data.fillna(0).assign(ctr=ad_data.click / ad_data.view)

In [11]:
top_ctr = ad_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)

In [12]:
top_ctr

event,ad_id,click,view,ctr
15,12904,32.0,399.0,0.080201
328,120347,236.0,2168.0,0.108856
5,4585,53.0,476.0,0.111345
96,35034,112.0,997.0,0.112337
194,45043,28.0,245.0,0.114286
207,45642,42.0,344.0,0.122093
144,40968,29.0,217.0,0.133641
125,38575,43.0,257.0,0.167315
324,119450,258.0,1254.0,0.205742
289,112583,580436.0,1934788.0,0.300000


In [13]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [15]:
path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-novohatnij-29/lesson 7/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)
json_data

{'file': '-216413731|0|-1|240331|5997162f25|csv|403|top_ctr_data.csv|c2f9e346fc49058f43500d24dbf7ae44|839ac716bb9c8fe8ebd863e80c22e5e1||||eyJkaXNrIjozOSwicGVlcl9zZW5kZXIiOiItMjE2NDEzNzMxIn0='}

In [16]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)

In [17]:
saved_file

{'type': 'doc',
 'doc': {'id': 645024227,
  'owner_id': -216413731,
  'title': 'top_ctr_data.csv',
  'size': 403,
  'ext': 'csv',
  'date': 1665392458,
  'type': 1,
  'url': 'https://vk.com/doc-216413731_645024227?hash=tZPb2wDevvt5o4OPZy0OngKG7tZKH6FGWuxuMq15Pho&dl=FUZDCNRUGEZTOMZR:1665392458:U2Ul4wZ4nBEQOT1anPjVqHOoM7HbtOlEKpyoyWm0R9X&api=1&no_preview=1'}}

In [18]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

In [19]:
attachment

'doc-216413731_645024227'

In [20]:
vk.messages.send(
    chat_id=1,
    random_id=3,
    message='Топ объявлений по CTR', 
    attachment = attachment
)

4

In [21]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [59]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

my_mail = 'anatoly1804@gmail.com'

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name('heroic-venture-268009-1dfbcc34e5fa.json', scope)
gs = gspread.authorize(credentials)

In [60]:
# Name of the table in google sheets, 
# can be url for open_by_url
# or id (key) part for open_by_key
table_name = 'to_sequence'  # Your table
# Get this table
work_sheet = gs.open(table_name)

In [72]:
# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()

In [73]:
data

[['price', 'group'],
 ['110', 'Group_1'],
 ['20', 'Group_2'],
 ['250', 'group_3'],
 ['500', 'group_4']]

In [74]:
headers = data.pop(0)

In [75]:
df = pd.DataFrame(data, columns=headers)

In [76]:
df

,price,group
0,110,Group_1
1,20,Group_2
2,250,group_3
3,500,group_4


In [67]:
df.sort_values('price', ascending=False)

,price,group
1,20,Group_2
0,110,Group_1


In [71]:
sheet1.append_row([500, 'group_4'])

{'spreadsheetId': '1gjyC_l8LMR3xq6h9w0GlbfsCpjl_V1j3dl9_0WxYyq8',
 'tableRange': "'Лист1'!A1:B4",
 'updates': {'spreadsheetId': '1gjyC_l8LMR3xq6h9w0GlbfsCpjl_V1j3dl9_0WxYyq8',
  'updatedRange': "'Лист1'!A5:B5",
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

In [77]:
# Looks like spreadsheet should be already present at the dist (so, run code in create table section)
spreadsheet_name = 'A new spreadsheet'
sheet = 'KarpovCorses2'
d2g.upload(df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'KarpovCorses2' id:1323350636>

In [22]:
url = 'https://api-metrika.yandex.net/stat/v1/data?'
visits = 'metrics=ym:s:visits&dimensions=ym:s:date&id=44147844'

In [23]:
vistis_url = url + visits

In [24]:
vistis_request = requests.get(vistis_url)

In [25]:
vistis_request

<Response [200]>

In [26]:
json_data = json.loads(vistis_request.text)

In [27]:
json_data['data']

[{'dimensions': [{'name': '2022-10-04'}], 'metrics': [778.0]},
 {'dimensions': [{'name': '2022-10-05'}], 'metrics': [721.0]},
 {'dimensions': [{'name': '2022-10-06'}], 'metrics': [707.0]},
 {'dimensions': [{'name': '2022-10-07'}], 'metrics': [692.0]},
 {'dimensions': [{'name': '2022-10-09'}], 'metrics': [361.0]},
 {'dimensions': [{'name': '2022-10-08'}], 'metrics': [296.0]},
 {'dimensions': [{'name': '2022-10-10'}], 'metrics': [281.0]}]

In [28]:
y_df = pd.DataFrame([(i['dimensions'][0]['name'], 
  i['metrics'][0]) for i in json_data['data']], columns=['date', 
                                                        'visits'])

In [29]:
spreadsheet_name = 'A new spreadsheet'
sheet = 'Yandex_visits'
d2g.upload(y_df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

NameError: name 'credentials' is not defined

In [33]:
vk.messages.send(
    chat_id=184706318,
    random_id=5,
    message='Это я, Почтальон Печкин!')

ApiError: [100] One of the parameters specified was missing or invalid: chat_id should be less or equal to 100000000